In [28]:
# Criar o navegador
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.alert import Alert
from selenium.webdriver import ActionChains
#from selenium.webdriver.common.keys import Keys
#from selenium.webdriver.support.ui import Select
from time import sleep

servico = Service(ChromeDriverManager().install()) # Identifica a versão do ChromeDriver e faz o download
navegador = webdriver.Chrome(service=servico) # Criar o navegador



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\dju0\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


In [29]:
# Abrir a página index (entrar no site da busca jurídica)
import os

caminho = os.getcwd()
arquivo = caminho + r"\index.html"

In [30]:
# importar a base de dados
import pandas as pd

tabela = pd.read_excel('Processos.xlsx') 
display(tabela)

,Nome,Advogado,Processo,Cidade,Status
0,Lira,Alon Lawyer,PC6592,Distrito Federal,NaN
1,João,Lawyer Alon,EB3792,Rio de Janeiro,NaN
2,Amanda,Amanda mesmo,MM1043,Rio de Janeiro,NaN
3,Carol,Amanda,PC5197,São Paulo,NaN


In [31]:
for linha in tabela.index:
    # Para cada processo (linha da tabela)
    navegador.get(arquivo)

    # Abrir a lista de cidades
    botao = navegador.find_element(By.CLASS_NAME, 'dropdown-menu') # depois testar com .get_attribute('href')
    ActionChains(navegador).move_to_element(botao).perform()

    cidade = tabela.loc[linha, 'Cidade']
    # Escolher o estado
    navegador.find_element(By.PARTIAL_LINK_TEXT, cidade ).click()

    # Mudar para nova aba
    aba_original = navegador.window_handles[0]
    indice = 1 + linha
    nova_aba = navegador.window_handles[indice]

    navegador.switch_to.window(nova_aba)

    # Preencher o formulário com os dados de busca
    
    # Nome da parte
    navegador.find_element(By.ID, 'nome').send_keys(tabela.loc[linha, 'Nome'])
    # Advogado
    navegador.find_element(By.ID, 'advogado').send_keys(tabela.loc[linha, 'Advogado'])
    # Número do processo
    navegador.find_element(By.ID, 'numero').send_keys(tabela.loc[linha, 'Processo'])

    # Clicar em pesquisar
    navegador.find_element(By.CLASS_NAME, 'registerbtn').click()

    # Confirmar a pesquisa
    alerta = navegador.switch_to.alert
    alerta.accept()

    # Esperar o resultado da pesquisa e agir de acordo com o resultado

    while True:
        try:
            alerta = navegador.switch_to.alert
            break
        except:
            sleep(1)
    texto_alerta = alerta.text

    if 'Processo encontrado com sucesso' in texto_alerta:
        alerta.accept()
        tabela.loc[linha, 'Status'] = 'Encontrado'
    else:
        tabela.loc[linha, 'Status'] = 'Não encontrado'
        alerta.accept()
    


In [33]:
navegador.quit()
display(tabela)

tabela.to_excel('Processos Atualizado.xlsx')

,Nome,Advogado,Processo,Cidade,Status
0,Lira,Alon Lawyer,PC6592,Distrito Federal,Encontrado
1,João,Lawyer Alon,EB3792,Rio de Janeiro,Encontrado
2,Amanda,Amanda mesmo,MM1043,Rio de Janeiro,Não encontrado
3,Carol,Amanda,PC5197,São Paulo,Não encontrado
